In [2]:
# =========================================================
# ✅ Step 1 — Install dependencies (Stable & Optimized for Kaggle, Oct 2025)
# =========================================================

# Upgrade pip
!pip install --upgrade --quiet pip

# --- 🤗 Hugging Face + optimization libraries ---
!pip install pyngrok flask --quiet
!pip install transformers -U

# --- 📦 Extra essentials for modern HF models ---
!pip install --quiet einops sentencepiece safetensors
!pip install flask-cors

import torch, torchvision, transformers, tokenizers

print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| cuDNN:", torch.backends.cudnn.version())
print("TorchVision:", torchvision.__version__)
print("Transformers:", transformers.__version__)
print("Tokenizers:", tokenizers.__version__)
print("GPUs:", torch.cuda.device_count(), [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 125.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 18.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.5 MB/s  0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc232m0/3 [huggingface-hub]
  Attempting uninstall: tokenizers━━━━━━━━━━━━━━ 0/3 [huggingface-hub]
    Found existing installation: tokenizers 0.21.20/3 [huggingface-hub]
    Uninstalling tokenizers-0.21.2:━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [tokenizers]
      Successfully uninstalled tokenizers-0.21.2━━━━━━━━━━━━━━ 1/3 [tokenizers]
  Attempting uninstall: transformers━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [tokenizers]
    Found existing installation: transformers 4.53.3━━━━━━━━━━ 1/3 [tokenizers]
  

In [3]:
# ======================
# Step 2 — Import Libraries
# ======================
from flask import Flask, request, render_template_string,Response,session,copy_current_request_context
from flask_cors import CORS
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline,AutoTokenizer,TextIteratorStreamer
from huggingface_hub import login
import threading
import time



2025-10-14 19:21:37.091313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760469697.314344      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760469697.381273      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Replace with your actual Hugging Face API token
HUGGINGFACE_API_KEY = "hf_your_actual_token_here"

# Login to Hugging Face
login(HUGGINGFACE_API_KEY)
print("Logged in to Hugging Face!")

Logged in to Hugging Face!


In [ ]:
# 👇 Replace this with your Ngrok auth token
NGROK_AUTH_TOKEN = "your_ngrok_auth_token"  # ← your token here

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [6]:
# =========================================================
# ✅ Load Qwen3-4B-Instruct-2507 model (Oct 2025)
#    — Extended context (262K tokens) + smart memory handling
# =========================================================
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-4B-Instruct-2507"

# Load tokenizer with full 262K token context
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    model_max_length=262144,   # match model's full context capacity
    use_fast=True,             # ensures the efficient Rust tokenizer backend
)

# Load model efficiently
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",         # automatically use bf16/fp16 depending on GPU
    device_map="auto",          # spread across multiple GPUs if available
    low_cpu_mem_usage=True,     # prevents CPU RAM spikes
)

# Confirm model context length
try:
    max_ctx = getattr(model.config, "max_position_embeddings", "unknown")
    print(f"✅ Model loaded: {model_name}")
    print(f"🧠 Max context window: {max_ctx} tokens")
except Exception as e:
    print(f"⚠️ Warning: could not read context size — {e}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

✅ Model loaded: Qwen/Qwen3-4B-Instruct-2507
🧠 Max context window: 262144 tokens


In [7]:
# top of your Flask file
from collections import defaultdict, deque
import threading

# Keep only the last N messages (user+assistant), not counting the system message
MAX_TURNS = 16
SESSIONS = defaultdict(lambda: deque(maxlen=MAX_TURNS))
SESSIONS_LOCK = threading.Lock()

In [13]:
# =========================================================
# 🚀 Minimal Flask Streaming API for Qwen-3-4B-Instruct
# =========================================================

# ---------------------------------------------------------
# ✅ Flask setup
# ---------------------------------------------------------
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True) # allow frontend requests


# ---------------------------------------------------------
# 🌐 /stream endpoint — core streaming generation
# ---------------------------------------------------------
@app.route("/stream", methods=["GET"])
def stream():
    # 1️⃣ Get the input text
    session_id = request.args.get("session_id", "").strip()
    user_input = request.args.get("prompt", "").strip()

    
    if not session_id:
            return Response(
                "data: Missing ?session_id query parameter.\n\ndata: [DONE]\n\n",
                mimetype="text/event-stream",
                headers={
                    "Access-Control-Allow-Origin": "*",
                    "Cache-Control": "no-cache",
                    "Connection": "keep-alive",
                    "X-Accel-Buffering": "no",
                },
            )

    if not user_input:
        return Response(
            "data: Please provide a ?prompt query parameter.\n\ndata: [DONE]\n\n",
            mimetype="text/event-stream",
            headers={"Access-Control-Allow-Origin": "*"}
        )

    
    # 1) Copy current history for this session (avoid holding the lock during generation)
    with SESSIONS_LOCK:
        history = list(SESSIONS[session_id])  # each item: {"role": "user"|"assistant", "content": str}
     # 🔎 Debug: log what the server sees
    print("SESSION:", session_id, "HISTORY LEN:", len(history))
    for i, m in enumerate(history[-4:], 1):
        print(f"  H[-{i}]:", m["role"], m["content"][:60])
    print("USER INPUT:", repr(user_input), flush=True)

    # 2️⃣ Create a chat-style prompt
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        *history,  # ← include prior turns
        {"role": "user", "content": user_input},
    ]

    #  3) Prepare prompt
    chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 4️⃣ Create a text streamer
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # 5️⃣ Tokenize + move to GPU
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    # 6️⃣ Generation settings
    generation_kwargs = dict(
        **inputs,
        streamer=streamer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    # 7️⃣ Start generation thread
    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # 8️⃣ Stream the output live
    def event_stream():
        
    # Collect full assistant text while streaming
        chunks = []
        # Optional: small prelude to defeat proxy buffering
        yield ":" + " " * 2048 + "\n\n"

        
        try:
            for new_text in streamer:
                chunks.append(new_text)
                yield f"data: {new_text}\n\n"
        finally:
            full_assistant = "".join(chunks)

            # 4) Persist the turn back into the session (user + assistant)
            with SESSIONS_LOCK:
                SESSIONS[session_id].append({"role": "user", "content": user_input})
                SESSIONS[session_id].append({"role": "assistant", "content": full_assistant})

            yield "data: [DONE]\n\n"


    return Response(event_stream(), mimetype="text/event-stream", 
                    headers={"Access-Control-Allow-Origin": "*",
                            "Cache-Control": "no-cache",
                            "Connection": "keep-alive",
                            "X-Accel-Buffering": "no"
})


In [9]:
# @app.route("/clear", methods=["POST"])
# def clear():
#     return "Chat cleared"


@app.route("/clear", methods=["POST"])
def clear():
    session_id = request.args.get("session_id", "").strip()
    with SESSIONS_LOCK:
        if session_id:
            SESSIONS.pop(session_id, None)
        else:
            # optional: clear every session (dev convenience)
            SESSIONS.clear()
    return "Chat cleared"

In [14]:
if __name__ == "__main__":
    ngrok.kill()
    public_url = ngrok.connect(5000)
    print(f"🌐 Public URL: {public_url.public_url}")
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)


🌐 Public URL: https://227298091052.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off
SESSION: test123 HISTORY LEN: 4
  H[-1]: user hello
  H[-2]: assistant Hello! How can I assist you today? 😊
  H[-3]: user hello
  H[-4]: assistant Hi again! 👋 How's it going? I'm here and ready to help—what'
USER INPUT: 'hello'
SESSION: test123 HISTORY LEN: 6
  H[-1]: user hello
  H[-2]: assistant Hi again! 👋 How's it going? I'm here and ready to help—what'
  H[-3]: user hello
  H[-4]: assistant Hey there! 🌟 Just wanted to say I'm happy to keep chatting w
USER INPUT: 'my name is joba'
SESSION: test123 HISTORY LEN: 8
  H[-1]: user hello
  H[-2]: assistant Hey there! 🌟 Just wanted to say I'm happy to keep chatting w
  H[-3]: user my name is joba
  H[-4]: assistant Hello Joba! 🌟 It's great to meet you! How can I assist you t
USER INPUT: 'what is my name'
SESSION: test123 HISTORY LEN: 10
  H[-1]: user my name is joba
  H[-2]: assistant Hello Joba! 🌟 It's great to meet you! How can I assist 